# 來源網址

https://www.cnblogs.com/huxi/archive/2010/07/04/1771073.html

**正則表達式--廖雪峰**

https://www.liaoxuefeng.com/wiki/001374738125095c955c1e6d8bb493182103fac9270762a000/001386832260566c26442c671fa489ebc6fe85badda25cd000

# 正則表達式

因为正则表达式也是用字符串表示的，所以，我们要首先了解如何用字符来描述字符。

在正则表达式中，如果直接给出字符，就是精确匹配。用`\d`可以匹配一个数字，`\w`可以匹配一个字母或数字，所以：

* '00\d'可以匹配'007'，但无法匹配'00A'；

* '\d\d\d'可以匹配'010'；
 
* '\w\w\d'可以匹配'py3'；

`.`可以匹配任意字符，所以：

* 'py.'可以匹配'pyc'、'pyo'、'py!'等等。

要匹配变长的字符，在正则表达式中，用`*`表示任意个字符（包括0个），用`+`表示至少一个字符，用`?`表示0个或1个字符，用`{n}`表示n个字符，用`{n,m}`表示n-m个字符：

来看一个复杂的例子：\d{3}\s+\d{3,8}。

我们来从左到右解读一下：

\d{3}表示匹配3个数字，例如'010'；

\s可以匹配一个空格（也包括Tab等空白符），所以\s+表示至少有一个空格，例如匹配' '，' '等；

\d{3,8}表示3-8个数字，例如'1234567'。

综合起来，上面的正则表达式可以匹配以任意个空格隔开的带区号的电话号码。

如果要匹配'010-12345'这样的号码呢？由于'-'是特殊字符，在正则表达式中，要用'\'转义，所以，上面的正则是`\d{3}\-\d{3,8}`。

但是，仍然无法匹配'010 - 12345'，因为带有空格。所以我们需要更复杂的匹配方式。

## 进阶

要做更精确地匹配，可以用`[]`表示范围，比如：

[0-9a-zA-Z\_]可以匹配一个数字、字母或者下划线；

[0-9a-zA-Z\_]+可以匹配至少由一个数字、字母或者下划线组成的字符串，比如'a100'，'0_Z'，'Py3000'等等；

[a-zA-Z\_][0-9a-zA-Z\_]*可以匹配由字母或下划线开头，后接任意个由一个数字、字母或者下划线组成的字符串，也就是Python合法的变量；

[a-zA-Z\_][0-9a-zA-Z\_]{0, 19}更精确地限制了变量的长度是1-20个字符（前面1个字符+后面最多19个字符）。

`A|B`可以匹配A或B，所以(P|p)ython可以匹配'Python'或者'python'。

`^`表示行的开头，`^\d`表示必须以数字开头。

`$`表示行的结束，`\d$`表示必须以数字结束。

你可能注意到了，py也可以匹配'python'，但是加上`^py$`就变成了整行匹配，就只能匹配'py'了。

## re模块


有了准备知识，我们就可以在Python中使用正则表达式了。Python提供re模块，包含所有正则表达式的功能。由于Python的字符串本身也用\转义，所以要特别注意：

```py
s = 'ABC\\-001' # Python的字符串
# 对应的正则表达式字符串变成：
# 'ABC\-001'
```

因此我们强烈建议使用Python的r前缀，就不用考虑转义的问题了：

```py
s = r'ABC\-001' # Python的字符串
# 对应的正则表达式字符串不变：
# 'ABC\-001'
```

先看看如何判断正则表达式是否匹配：

```py
import re
re.match(r'^\d{3}\-\d{3,8}$', '010-12345')
<_sre.SRE_Match object at 0x1026e18b8>

re.match(r'^\d{3}\-\d{3,8}$', '010 12345')
```

match()方法判断是否匹配，如果匹配成功，返回一个Match对象，否则返回None。常见的判断方法就是：

test = '用户输入的字符串'
if re.match(r'正则表达式', test):
    print 'ok'
else:
    print 'failed'


### 切分字符串

```py
>>> 'a b   c'.split(' ')
['a', 'b', '', '', 'c']
```

嗯，无法识别连续的空格，用正则表达式试试：

```py
>>> re.split(r'\s+', 'a b   c')
['a', 'b', 'c']
```

无论多少个空格都可以正常分割。加入,试试：

```py
>>> re.split(r'[\s\,]+', 'a,b, c  d')
['a', 'b', 'c', 'd']
```

再加入;试试：

```py
>>> re.split(r'[\s\,\;]+', 'a,b;; c  d')
['a', 'b', 'c', 'd']
```

如果用户输入了一组标签，下次记得用正则表达式来把不规范的输入转化成正确的数组。

### 分组


除了简单地判断是否匹配之外，正则表达式还有提取子串的强大功能。用()表示的就是要提取的分组（Group）。比如：

`^(\d{3})-(\d{3,8})$`分别定义了两个组，可以直接从匹配的字符串中提取出区号和本地号码：

In [6]:
import re

m = re.match(r'^(\d{3})-(\d{3,8})$', '010-12345')
m

m.group(0)

m.group(1)

m.group(2)


<_sre.SRE_Match object; span=(0, 9), match='010-12345'>

'010-12345'

'010'

'12345'

如果正则表达式中定义了组，就可以在Match对象上用group()方法提取出子串来。

注意到group(0)永远是原始字符串，group(1)、group(2)……表示第1、2、……个子串。

### 貪婪匹配、非貪婪匹配

最后需要特别指出的是，正则匹配默认是贪婪匹配，也就是匹配尽可能多的字符。举例如下，匹配出数字后面的0：

In [8]:
re.match(r'^(\d+)(0*)$', '102300').groups()

('102300', '')

由于`\d+`采用贪婪匹配，直接把后面的0全部匹配了，结果`0*`只能匹配空字符串了。

必须让`\d+`采用非贪婪匹配（也就是尽可能少匹配），才能把后面的0匹配出来，加个`?`就可以让\d+采用非贪婪匹配：

In [9]:
re.match(r'^(\d+?)(0*)$', '102300').groups()

('1023', '00')

### 编译

当我们在Python中使用正则表达式时，re模块内部会干两件事情：

编译正则表达式，如果正则表达式的字符串本身不合法，会报错；

用编译后的正则表达式去匹配字符串。

如果一个正则表达式要重复使用几千次，出于效率的考虑，我们可以预编译该正则表达式，接下来重复使用时就不需要编译这个步骤了，直接匹配：

In [13]:
import re
# 编译:
re_telephone = re.compile(r'^(\d{3})-(\d{3,8})$')
# 使用：
re_telephone.match('010-12345').groups()

re_telephone.match('010-8086').groups()


('010', '12345')

('010', '8086')

## 小節--小試身手

请尝试写一个验证Email地址的正则表达式。版本一应该可以验证出类似的Email：

someone@gmail\.com

bill.gates@microsoft\.com

版本二可以验证并提取出带名字的Email地址：

`<Tom Paris> tom@voyager.org`

说明一下：

`^[0-9a-zA-Z][0-9a-zA-Z\_\.]+\@[0-9a-z]+\.(com|cn|org|net)$`

第一部分 `^[0-9a-zA-Z]` 邮箱基本上是以数字或字母为开头；

第二部分 `[0-9a-zA-Z\_\.]+` 后面可以是数字、字母、下划线、小数点（目前了解是这样，可能有些邮箱比较特殊）；

第三部分 `[0-9a-z]+` 域名前半部分通常都是小写字母或数字，如 gmail、163；

第四部分 `(com|cn|org|net)$ `域名后半部分基本上是以这几个为结尾，还有 edu、gov 等暂未考虑进去。

所以，整个正则表达式还可以简写成 `^[\w][\w\_\.]+\@[\w]+\.(com|cn|org|net)$`

In [35]:
email = '<Tom Paris> tom@voyager.org'

import re

re_email = re.compile(r'(^[0-9a-zA-Z][0-9a-zA-Z\_\.]+)\@[0-9a-z]+\.(com|cn|org|net)$')

m = re_email.match(email)

if m:
    print("OK.")
else:
    print("You have input a wrong e-mail.")

You have input a wrong e-mail.


In [34]:
m.group(1)

'<Tom Paris> tom'

# pandas str

In [117]:
import requests

import pandas as pd

In [118]:
url = 'https://www.cmegroup.com/CmeWS/mvc/Settlements/Futures/Settlements/437/FUT'
#?strategy=DEFAULT&tradeDate=03%2F20%2F2019&pageSize=50&_=1553246305853'

data = {
    'strategy': 'DEFAULT',
    'tradeDate': '03/21/2019'
}

res = requests.get(url, data)
res.json().keys() #['settlements']

dict_keys(['settlements', 'updateTime', 'dsHeader', 'reportType', 'tradeDate', 'empty'])

In [119]:
df = pd.DataFrame(res.json()['settlements'])
df = df[df != '-'].dropna()
df

,change,high,last,low,month,open,openInterest,settle,volume
1,+5.6,1320.2,1309.0,1302.8,APR 19,1312.7,"201,837",1307.3,"333,112"
2,+5.9,1322.4,1312.1,1306.6,MAY 19,1316.0,978,1310.6,"1,398"
3,+5.8,1326.3,1315.2,1309.1,JUN 19,1318.6,"219,795",1313.6,"62,519"
4,+5.7,1332.2,1321.3,1316.0,AUG 19,1326.1,"43,342",1319.7,"5,240"
5,+5.7,1336.7,1327.1,1322.0,OCT 19,1332.7,"5,790",1325.5,131
6,+5.8,1344.2,1331.7,1327.8,DEC 19,1338.0,"28,287",1331.6,"1,301"
7,+5.5,1348.0,1336.0,1333.1,FEB 20,1348.0,"9,783",1337.4,206
8,+5.5,1355.7B,1349.9,1349.9,APR 20,1352.2,"2,992",1343.2,145
9,+5.5,1357.9,1357.9,1357.9,JUN 20,1357.9,"1,592",1349.0,117
21,,,,,Total,,"516,116",,"404,169"


In [120]:
df.index = df['month']

df1 = df.iloc[:9,:].loc[:,['open','high','low','last','openInterest']][:]
df1

,open,high,low,last,openInterest
month,,,,,
APR 19,1312.7,1320.2,1302.8,1309.0,"201,837"
MAY 19,1316.0,1322.4,1306.6,1312.1,978
JUN 19,1318.6,1326.3,1309.1,1315.2,"219,795"
AUG 19,1326.1,1332.2,1316.0,1321.3,"43,342"
OCT 19,1332.7,1336.7,1322.0,1327.1,"5,790"
DEC 19,1338.0,1344.2,1327.8,1331.7,"28,287"
FEB 20,1348.0,1348.0,1333.1,1336.0,"9,783"
APR 20,1352.2,1355.7B,1349.9,1349.9,"2,992"
JUN 20,1357.9,1357.9,1357.9,1357.9,"1,592"


In [122]:
df1['openInterest'].apply(
    lambda x:x.replace(',','')
)

month
APR 19    201837
MAY 19       978
JUN 19    219795
AUG 19     43342
OCT 19      5790
DEC 19     28287
FEB 20      9783
APR 20      2992
JUN 20      1592
Name: openInterest, dtype: object

In [123]:
df1['open'].str.replace(',','').astype(float)

month
APR 19    1312.7
MAY 19    1316.0
JUN 19    1318.6
AUG 19    1326.1
OCT 19    1332.7
DEC 19    1338.0
FEB 20    1348.0
APR 20    1352.2
JUN 20    1357.9
Name: open, dtype: float64

In [113]:
#['high'] 有時會出現 1355.7B 用正則篩選

df1['high'].str.extract(r'(\d*\.\d*)').astype(float) +90000

,0
month,
APR 19,91320.2
MAY 19,91322.4
JUN 19,91326.3
AUG 19,91332.2
OCT 19,91336.7
DEC 19,91344.2
FEB 20,91348.0
APR 20,91355.7
JUN 20,91357.9


In [112]:

# r''正則 |:表 "或" PS:不能有空白
# 正則
#https://www.cnblogs.com/huxi/archive/2010/07/04/1771073.html
    
# .str.extract(r'(\d*\.\d*|\d*)',expand=False
# expand=False 返回 序列(Series)
#https://blog.csdn.net/claroja/article/details/64929819

df1.apply(
    lambda x:x.str.replace(',','').str.extract(r'(\d*\.\d*|\d*)',expand=False) 
).astype(float)

Index(['APR 19', 'MAY 19', 'JUN 19', 'AUG 19', 'OCT 19', 'DEC 19', 'FEB 20',
       'APR 20', 'JUN 20'],
      dtype='object', name='month')

,open,high,low,last,openInterest
month,,,,,
APR 19,1312.7,1320.2,1302.8,1309.0,201837.0
MAY 19,1316.0,1322.4,1306.6,1312.1,978.0
JUN 19,1318.6,1326.3,1309.1,1315.2,219795.0
AUG 19,1326.1,1332.2,1316.0,1321.3,43342.0
OCT 19,1332.7,1336.7,1322.0,1327.1,5790.0
DEC 19,1338.0,1344.2,1327.8,1331.7,28287.0
FEB 20,1348.0,1348.0,1333.1,1336.0,9783.0
APR 20,1352.2,1355.7,1349.9,1349.9,2992.0
JUN 20,1357.9,1357.9,1357.9,1357.9,1592.0


In [125]:
s = 'Tuesday, 26 Mar 2019 10:33 PM'

In [129]:
s.split(' ')

['Tuesday,', '26', 'Mar', '2019', '10:33', 'PM']

In [130]:
s.split(' ')[1:4]

['26', 'Mar', '2019']

In [131]:
import re

In [135]:
temp= r"\d{1,2} [a-zA-Z]{3} \d{4}"  
re.search(temp, s)

<_sre.SRE_Match object; span=(9, 20), match='26 Mar 2019'>